In [ ]:
# Self-Supervised Learning, vs. semi- or unsupverised

In [ ]:
# Importance of ViT, DINOv2 Approach

In [ ]:
# Emerging properties of DINOv2

In [ ]:
# LoRA finetuning, significant improvement

In [ ]:
# Semantic Segmentation

In [ ]:
# Instance segmentation